In [ ]:
#NBVAL_SKIP
import matplotlib.pyplot as plt
from rubix.core.pipeline import RubixPipeline 
import os
config = {
    "pipeline":{"name": "calc_ifu"},
    
    "logger": {
        "log_level": "DEBUG",
        "log_file_path": None,
        "format": "%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    },
    "data": {
        "name": "IllustrisAPI",
        "args": {
            "api_key": os.environ.get("ILLUSTRIS_API_KEY"),
            "particle_type": ["stars", "gas"],
            #"cube_type": ["stars"],
            "simulation": "TNG50-1",
            "snapshot": 99,
            "save_data_path": "data",
        },
        
        "load_galaxy_args": {
        "id": 11,
        "reuse": True,
        },
        
        "subset": {
            "use_subset": False,
            "subset_size": 1000,
        },
    },
    "simulation": {
        "name": "IllustrisTNG",
        "args": {
            "path": "data/galaxy-id-11.hdf5",
        },
    
    },
    "output_path": "output",

    "telescope":
        {"name": "MUSE",
         "psf": {"name": "gaussian", "size": 5, "sigma": 0.6},
         "lsf": {"sigma": 0.5},
         "noise": {"signal_to_noise": 100,"noise_distribution": "normal"},},
    "cosmology":
        {"name": "PLANCK15"},
        
    "galaxy":
        {"dist_z": 0.1,
         "rotation": {"type": "edge-on"},
        },
        
    "ssp": {
        "template": {
            "name": "BruzualCharlot2003"
        },
    },        
}

pipe = RubixPipeline(config)

data= pipe.run()

In [ ]:
datacube = data.stars.datacube
img = datacube.sum(axis=2)
plt.imshow(img, origin="lower")

In [ ]:
import jax.numpy as jnp

#NBVAL_SKIP
wavelengths = pipe.telescope.wave_seq

# get the indices of the visible wavelengths of 4000-8000 Angstroms

visible_indices = jnp.where((wavelengths >= 4000) & (wavelengths <= 8000))

spec = datacube[12, 12]

plt.plot(wavelengths[visible_indices], spec[visible_indices])

In [ ]:
from mpdaf.obj import Cube
from astropy.io import fits

cube = Cube(filename="../../ESO079-003_DATACUBE_iDR1.fits")
header = cube.primary_header.copy()

hdu = fits.PrimaryHDU(data=None, header=header)
hdu.writeto("output/MUSE_example_header.fits", overwrite=True)

In [ ]:
from datetime import datetime
from astropy.time import Time

comment_keywords = [key for key in header.keys() if key == 'COMMENT']

for key in comment_keywords:
    header.remove(key)

header.remove('HIERARCH MPDAF METH1 VERSION')
header.remove('HIERARCH MPDAF METH1 ID')
header.remove('HIERARCH MPDAF METH1 PARAM1 NAME')
header.remove('HIERARCH MPDAF METH1 PARAM1 VALUE')
header.remove('HIERARCH MPDAF METH1 PARAM2 NAME')
header.remove('HIERARCH MPDAF METH1 PARAM2 VALUE')
header.remove('HIERARCH MPDAF METH1 PARAM3 NAME')
header.remove('HIERARCH MPDAF METH1 PARAM3 VALUE')
header.remove('HIERARCH MPDAF METH1 PARAM4 NAME')
header.remove('HIERARCH MPDAF METH1 PARAM4 VALUE')
header.remove('HIERARCH MPDAF METH1 PARAM5 NAME')
header.remove('HIERARCH MPDAF METH1 PARAM5 VALUE')
header.remove('NFILES')
header.remove('DATE')
header.remove('AUTHOR')

# Comments
header['COMMENT'] = 'This datacube was calculated using the RUBIX pipeline'
header['COMMENT'] = f"Input: {config['data']['name']} id {config['data']['load_galaxy_args']['id']} snapshot {config['data']['args']['snapshot']}"
header['COMMENT'] = f"{config['telescope']['name']} telescope configuration, {config['galaxy']['rotation']['type']} rotation, z={config['galaxy']['dist_z']}"
header['COMMENT'] = f"SSP lookup: {config['ssp']['template']['name']}"
header['COMMENT'] = f"Subset of stars used: {config['data']['subset']['use_subset']}"
header['COMMENT'] = 'This cube only contains stellar spectra!'
header['COMMENT'] = f"PSF: {config['telescope']['psf']['name'].capitalize()}, size {config['telescope']['psf']['size']}, sigma {config['telescope']['psf']['sigma']}"
header['COMMENT'] = f"LSF: Gaussian, sigma {config['telescope']['lsf']['sigma']}"
header['COMMENT'] = f"Noise: Signal to noise {config['telescope']['noise']['signal_to_noise']}, noise distribution {config['telescope']['noise']['noise_distribution']}"

# Data-specific fields
header['BITPIX'] = -32  # because datacube.dtype is float32
header['ORIGIN'] = 'AstroAI-Lab'
header['TELESCOP'] = 'RUBIX'
header['OBJECT'] = f"{config['simulation']['name']} id {config['data']['load_galaxy_args']['id']}"

# Add some default values for RA, DEC, EXPTIME
header['RA'] = 0.0
header['DEC'] = 0.0
header['EXPTIME'] = 10.0  # Assuming an exposure time of 10 seconds

# Date and MJD
now = datetime.now()
observation_time = now.strftime('%Y-%m-%dT%H:%M:%S.%f')[:-3]
t = Time(observation_time, format='isot', scale='utc')
mjd_obs = t.mjd

header['MJD-OBS'] = mjd_obs
header['DATE-OBS'] = observation_time


In [ ]:
header

In [ ]:
hdu = fits.PrimaryHDU(data=None, header=header)
#hdu.writeto(f"output/{config['simulation']['name']}id{config['data']['load_galaxy_args']['id']}_{config['data']['args']['cube_type'][0]}.fits", overwrite=True)
hdu.writeto(f"output/{config['simulation']['name']}id{config['data']['load_galaxy_args']['id']}_stars.fits", overwrite=True)